In [1]:
pip install opencv-python opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install scikit-image

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install scikit-learn

In [2]:
pip install numpy opencv-python pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow

In [ ]:
pip install torch torchvision

In [1]:
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Scaling factor to resize the video frames
scale_factor = 0.5

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX
org_text = (10, 30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization with labels
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Display the combined visualization with file name and quit instructions
    file_name = video_path.split("\\")[-1]
    cv2.putText(combined, f'File: {file_name}', (10, 50), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(combined, 'Press "q" to quit', (10, 80), font, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Classic Methods Visualization', combined)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Properly release the video and close the window
cap.release()
cv2.destroyAllWindows()

End of video.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade opencv-python opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
#To save the processed video
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Output video file path
output_path = r"D:\\Sem-4\\Project\\Processed_Output.avi"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for the output video

# Scale down the resolution for processing
scale_factor = 0.5
output_width = int(frame_width * scale_factor)
output_height = int(frame_height * scale_factor)

# Initialize VideoWriter for saving the processed video
out = cv2.VideoWriter(output_path, fourcc, fps, (output_width * 3, output_height * 3))  # 3x3 grid

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, (output_width, output_height), interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Write to the output video
    out.write(combined)

# Properly release the video and writer
cap.release()
out.release()
print(f"Processing complete. Saved to {output_path}.")


End of video.
Processing complete. Saved to D:\\Sem-4\\Project\\Processed_Output.avi.


In [4]:
import cv2
import numpy as np

# Input video file path
video_path = r"D:\\Sem-4\\Project\\Datasets\\VISEM-Tracking - Datasets from Paper 2\\VISEM_Tracking_Train_v4\\Train\\11\\11.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Scaling factor to resize the video frames
scale_factor = 0.5

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Overlay text
font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Resize the frame
    frame_resized = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 1. Grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # 2. Background Subtraction
    foreground_mask = bg_subtractor.apply(frame_resized)

    # 3. Thresholding
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # 4. Morphological Operations
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # Opening
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # Closing

    # 5. Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # 6. Blurring
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 7. Contour Detection
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_frame = frame_resized.copy()
    cv2.drawContours(contour_frame, contours, -1, (0, 255, 0), 2)

    # Combine all images into a single visualization
    top_row = np.hstack([
        cv2.putText(frame_resized.copy(), 'Original', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), 'Grayscale', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR), 'Foreground', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    middle_row = np.hstack([
        cv2.putText(cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR), 'Threshold', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(opened, cv2.COLOR_GRAY2BGR), 'Opened', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR), 'Closed', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])
    bottom_row = np.hstack([
        cv2.putText(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 'Edges', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR), 'Blurred', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA),
        cv2.putText(contour_frame, 'Contours', (10, 30), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    ])

    # Stack the combined images
    combined = np.vstack([top_row, middle_row, bottom_row])

    # Display the combined visualization
    cv2.imshow('Classic Methods Visualization', combined)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Properly release the video and close the window
cap.release()
cv2.destroyAllWindows()
